## Instructions

Load the [dataset]('https://drive.google.com/file/d/1OtwemNuw0qilXAutyuZEjOscZVKuD7Tv/view').

- The file has no index column.
- The last column is the target column.
- The first row of the file has column ids.



In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('NPPE1_ModelBuilding3.csv')
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.087472,0.002829,1.768235,0.188396,0.117071,0.148148,0.478165,0.720443,0.967195,0.281563,0.428571,0.959596,1.0,0.0,23.358097
1,0.378379,0.022079,1.115629,0.091974,0.066089,0.851852,0.911759,0.785321,0.885001,0.424648,0.285714,0.868687,0.0,1.0,17.268768
2,0.066901,0.003828,-0.536262,0.221188,0.255671,0.296296,0.228024,0.406472,0.980184,0.274376,0.428571,0.767677,0.0,1.0,27.776974
3,0.140645,0.011132,1.323366,0.422514,0.153103,0.148148,0.410679,0.200319,0.861371,0.305006,0.142857,0.848485,0.0,1.0,16.121960
4,0.144225,0.204918,-0.930790,0.148694,0.177490,0.259259,0.146832,0.111429,0.983448,0.286322,0.285714,0.616162,1.0,0.0,23.129426


# Q1

---

Which dataset are you using for this exam?

NPPE1_ModelBuilding1.csv

NPPE1_ModelBuilding2.csv

[x] NPPE1_ModelBuilding3.csv

NPPE1_ModelBuilding4.csv

In [4]:
"NPPE1_ModelBuilding3.csv"

'NPPE1_ModelBuilding3.csv'

# Q2

Split the dataset into train dataset and test dataset in the following manner:

- Use train_test_split to split the dataset into train and test dataset with test size equal to 0.3(i.e.30%) and random_state equal to 42. Let other parameters have default values.
- Columns except the last column should be the feature matrix (X_train or X_test)
- Last column will be the label vector.


In [5]:
# iloc - df[start_row_idx end_row_idx, start_col_idx end_col_idx]
# create feature and labels
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
X.shape, y.shape

((4000, 14), (4000,))

In [7]:
# train test split
from sklearn.model_selection import train_test_split

X_train , X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 42)

X_train, y_train

((1200, 14), (1200,))

Train the ridge model on the training data with the following parameters:

alpha = 10
solver = 'saga'
tol = 1e-4
random_state = 42

In [9]:
from sklearn.linear_model import Ridge

model_ridge = Ridge(alpha = 10,
                   solver = 'saga',
                   tol = 1e-4,
                   random_state = 42)

model_ridge.fit(X_train, y_train)

Ridge(alpha=10, random_state=42, solver='saga')

In [14]:
# check r2 scores
from sklearn.metrics import r2_score
#1
model_ridge.score(X_test , y_test)

#2
y_pred_ridge = model_ridge.predict(X_test)

r2_score(y_test, y_pred_ridge)

0.6613547575262211

# Q3


What is the index of most important feature? Note the index starts from 0. Ignore the intercept for this question.

In [18]:
# get feature importance index
model_ridge.coef_ # 9

array([ -0.4825441 ,   3.74601838,  -0.73583331,   0.54199933,
        -9.89014109,   5.80114296,  -5.06099736,  -9.45015598,
         4.73124885, -23.51321982,  11.31863371,   0.49450664,
        -0.89196134,   0.89196134])

# Q4

What is the index of least important feature? Note the index starts from 0. Ignore the intercept for this question.

In [19]:
model_ridge.coef_ #0

array([ -0.4825441 ,   3.74601838,  -0.73583331,   0.54199933,
        -9.89014109,   5.80114296,  -5.06099736,  -9.45015598,
         4.73124885, -23.51321982,  11.31863371,   0.49450664,
        -0.89196134,   0.89196134])

Take Only absolute values - so even if negative , convert to pos and then check

---

# Instructions

Take SGDRegressor(random_state = 42) estimator with GridSearchCV.

Hyperparameter tuning to be done over the following parameters:
- penalty as ['l1', 'l2']
- alpha values as [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
- values of tol as [1e-4, 1e-3, 1e-2, 1e-1]
- Use cross-validation = 5
- Set scoring as neg_mean_absolute_error

Use the best model from above hyper parameter tuning process to answer upcomming questions

In [20]:
# imports
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV

In [22]:
# creating instance
model_sgd = SGDRegressor(random_state = 42)

# creating params grid
params_grid = {
    "penalty" : ['l1', 'l2'],
    "alpha": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    "tol": [1e-4, 1e-3, 1e-2, 1e-1]
}

# intantiate grid search object
grid_seach = GridSearchCV(
    model_sgd,
    params_grid,
    cv = 5,
    scoring = "neg_mean_absolute_error"
)

# run the search
grid_seach.fit(X_train, y_train)

# store best model
best_sgd_model = grid_seach.best_estimator_
best_sgd_model

SGDRegressor(alpha=0.001, random_state=42, tol=0.0001)

# Q5

What is the best penalty?

l1

l2

In [24]:
# use get_params()[parameter_name]
best_sgd_model.get_params()['penalty']

'l2'

# Q6

What will be value of mean absolute error on the test dataset?

In [27]:
# get mean absolute error on dataset
from sklearn.metrics import mean_absolute_error

y_pred_sgd_best = best_sgd_model.predict(X_test)

mean_absolute_error(y_test , y_pred_sgd_best)

3.8131121797994014

---

# Instructions

Create a pipeline of the PCA() as transformer and Lasso as an estimator.

Use GridSearchCV for tuning the hyperparameters of the created pipeline on training dataset.
- Values of n_components for PCA to be [0.9, 0.95]
- lasso alpha value to be taken as : [10, 1, 0.01, 0.001]
- scoring : neg_mean_absolute_error.
- Use 5 fold cross validation.
- n_jobs = -1 (negative one) [it helps in using all the computational power to run this job]

(Note: Kindly ignore the warning.)


In [28]:
# imports
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [30]:
# pipline object
pipeline = Pipeline([
    ("pca", PCA()),
    ("lasso",Lasso())
])
pipeline

Pipeline(steps=[('pca', PCA()), ('lasso', Lasso())])

In [35]:
# create prarams grid
params_grid = {
    'pca__n_components' : [0.9, 0.95],
    'lasso__alpha' : [10, 1, 0.01, 0.001]
}

# intantiate grid search object
grid_seach = GridSearchCV(
    pipeline,
    params_grid,
    cv = 5,
    scoring = "neg_mean_absolute_error",
    n_jobs = -1
)

# start grid search
grid_seach.fit(X_train, y_train)

# store best model
best_pca_lasso_model = grid_seach.best_estimator_
best_pca_lasso_model

Pipeline(steps=[('pca', PCA(n_components=0.95)), ('lasso', Lasso(alpha=0.01))])

# Q7

If we fit the pipeline on the training dataset, what will be the R2 score on the test dataset?


In [37]:
best_pca_lasso_model.fit(X_train , y_train)
best_pca_lasso_model.score(X_test , y_test)

0.6288625430197559

# Q8

How much variance is explained (Eigen value) by the first principle component?

In [43]:
# get variance explained by 1st principle component
best_pca_lasso_model[0].explained_variance_[0]

1.1635075742239036

---

# Instructions

Create a pipeline of the PolynomialFeatures as transformer and Lasso as an estimator with the following parameters:
  - For PolynomialFeatures:
    - interaction_only = False
    - degree = 2
  - For Lasso:
    - alpha = 1
    - warm_start = True
    - 
    - random state as 0

In [44]:
# imports
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline

In [47]:
# create pipeline
pipeline = Pipeline([
    ("poly", PolynomialFeatures(interaction_only = False, degree = 2 )),
    ("lasso",Lasso(alpha = 1, warm_start = True, random_state = 0))
])
pipeline

Pipeline(steps=[('poly', PolynomialFeatures()),
                ('lasso', Lasso(alpha=1, random_state=0, warm_start=True))])

# Q9

Fit the pipeline on the training dataset and find the R2
score on the test dataset.

In [49]:
# fit the pipline on train dataset
pipeline.fit(X_train , y_train)

pipeline.score(X_test , y_test)

0.15767799891021772

In [51]:
y_pred = pipeline.predict(X_test)

r2_score(y_test , y_pred)

0.15767799891021772

---

# Q10

If you eliminate 1 feature with recursive feature elimination, which feature will be eliminated?

Type the index of the eliminated feature (index starts from 0).

- Use LinearRegression model with default parameters as an estimator.
- Use processed training data.

In [52]:
# import RFE
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [53]:
# lin reg
model_lin_reg = LinearRegression() # estimator

# rfe
# eliminate 1 feature with recursive feature elimination,
rfe = RFE(model_lin_reg, n_features_to_select = X_train.shape[1]-1)

# fit rfe
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=13)

In [54]:
# simple
rfe.ranking_

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [55]:
# hard
el_fit_idx = list(rfe.ranking_).index(max(rfe.ranking_))
el_fit_idx

2